In [ ]:
# source:
# https://medium.com/check-visit-computer-vision/understanding-360-images-8e0fcf0ee861

In [2]:
import numpy as np
from typing import List
import cv2
import numpy as np

In [5]:
def sample_pixels(begin: float, end: float, n: int) -> np.ndarray:
    """
    Sample pixel values of an affine function, which is equal
    to "begin" on the far left and "end" on the far right.
    """
    x_coords = 0.5 + np.arange(n, dtype=np.float32)
    return begin + (end-begin) * x_coords / n  # ]begin, end[

In [4]:
def equirec_to_cubemap(equirec: np.ndarray, out_size: int) -> List[np.ndarray]:
    """Convert an equirectangular image to a list of cubefaces (FRBLUD)"""
    height, width = equirec.shape[:2]

    u, v = np.meshgrid(sample_pixels(-1, 1, out_size),
                       sample_pixels(-1, 1, out_size),
                       indexing="ij")
    ones = np.ones((out_size, out_size), dtype=np.float32)

    list_xyz = [
        (v, u, ones),    # FRONT
        (ones, u, -v),   # RIGHT
        (-v, u, -ones),  # BACK
        (-ones, u, v),   # LEFT
        (v, -ones, u),   # UP
        (v, ones, -u)    # DOWN
    ]

    faces = []
    r = np.sqrt(u**2 + v**2 + 1) # Same values for each face
    for x, y, z in list_xyz:
        # Camera Convention RIGHT_DOWN_FRONT
        phi = np.arcsin(y/r)  # in [-pi/2, pi/2]
        theta = np.arctan2(x, z)  # in [-pi, pi]

        phi_map = (phi/np.pi + 0.5) * height
        theta_map = (theta/(2*np.pi) + 0.5) * width
    
        # Opencv shift
        theta_map -= 0.5
        phi_map -= 0.5
        
        faces.append(cv2.remap(equirec, theta_map, phi_map, cv2.INTER_CUBIC,
                               borderMode=cv2.BORDER_WRAP))
    return faces

In [ ]:
img = cv2.imread("img/room.jpg")

faces = equirec_to_cubemap(img, 512)

for i, face in enumerate(faces):
    cv2.imwrite(f"img/face_{i}.jpg", face)

print("Exported cubemap")

error: OpenCV(4.11.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:836: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'
